In [41]:
from pathlib import Path
import sys

ROOT_DIR = Path().resolve().parent
sys.path.append(str(ROOT_DIR))

from src.reading_data import df_brazil_responses, df_painel_agency

In [42]:
df_brazil_responses = df_brazil_responses.ffill()
df_brazil_responses.head(5)

,President,Year,Category (Bureaucracy or cabinet),Agency,Party,Name of Authority,Total Months (if not the whole year),Conceded Response,Partially Conceded Response,Denied,Others,Conceded and Partially conceded,Total Requests,% Conceded,% Partially conceded,% denied,% Others,% Concedico e Parcialmente
0,Dilma Rousseff,2015.0,Burocracia,ABGF - Agencia Brasileira Gestora de Fundos Ga...,NaN,NaN,NaN,18.0,NaN,NaN,35.0,18,53,0.339623,0.000000,0.0,0.660377,0.339623
1,Dilma Rousseff,2015.0,Burocracia,AEB – Agência Espacial Brasileira,NaN,NaN,NaN,48.0,3.0,NaN,2.0,51,53,0.905660,0.056604,0.0,0.037736,0.962264
2,Dilma Rousseff,2015.0,Burocracia,AMAZUL - Amazônia Azul Tecnologias de Defesa S.A.,NaN,NaN,NaN,10.0,3.0,NaN,1.0,10,11,0.909091,0.000000,0.0,0.090909,0.909091
3,Dilma Rousseff,2015.0,Burocracia,AN – Arquivo Nacional,NaN,NaN,NaN,147.0,2.0,NaN,7.0,149,156,0.942308,0.012821,0.0,0.044872,0.955128
4,Dilma Rousseff,2015.0,Burocracia,ANA – Agência Nacional de Águas,NaN,NaN,NaN,318.0,1.0,NaN,1.0,319,320,0.993750,0.003125,0.0,0.003125,0.996875


In [33]:
df_painel_agency.head(5)

,sigla_new,year,type_of_institution,n_recursos,n_conspicuous_nc,n_stonewalling,poc_conspicuous_nc_pedidos,porc_stonewalling_pedidos,orgao_cod,orgao,orgao_superior,orgao_superior_cod,despesa_total,receita_arrecadada_liquida,orgao_siape,n_funcionarios_mean,n_ouvidor_mean,n_emprestado_in_mean,n_cargo_decisao_mean,n_filiados_mean,n_inconclusivo_mean,n_filiados_cg_decisao_mean,n_inconclusivo_cg_decisao_mean,salario_mean,salario_median_mean,n_salario_na_mean,salario_cg_decisao_mean,partidos,partidos_cg_decisao,partido_mais_frequente,partido_cg_decisao_mais_frequente,n_pedidos,n_recurso_1instancia_deferido,n_recurso_1instancia_nao_conhecimento,n_recurso_1instancia_indeferido,n_recurso_1instancia_parcial_deferido,n_recurso_reclamacao_NA,n_recurso_2instancia_deferido,n_recurso_2instancia_indeferido,n_recurso_2instancia_parcial_deferido,n_recurso_2instancia_nao_conhecimento,n_recurso_1instancia_perda_objeto,n_recurso_1instancia_NA,n_recurso_2instancia_NA,n_recurso_2instancia_perda_objeto,n_recurso_cgu_deferido,n_recurso_cgu_indeferido,n_recurso_cgu_nao_conhecimento,n_recurso_cgu_parcial_deferido,n_recurso_cgu_perda_objeto,n_recurso_cmri_NA,n_recurso_cmri_deferido,n_recurso_cmri_indeferido,n_recurso_cmri_nao_conhecimento,n_recurso_cmri_perda_objeto,n_recurso_cgu_perda_objeto_parcial,n_recurso_cmri_parcial_deferido,n_recurso_1instancia_perda_objeto_parcial,n_recurso_cgu_acolhimento,n_recurso_revisao_deferido,n_recurso_2instancia_perda_objeto_parcial,n_recurso_cgu_NA,n_recurso_cmri_perda_objeto_parcial,n_recurso_revisao_indeferido,n_recurso_3instancia_deferido,n_recurso_3instancia_NA,n_recurso_3instancia_nao_conhecimento,n_recurso_3instancia_indeferido,n_recurso_3instancia_perda_objeto,n_recurso_4instancia_deferido,n_recurso_4instancia_nao_conhecimento,n_recurso_revisao_NA,n_recurso_3instancia_parcial_deferido,n_recurso_4instancia_indeferido
0,agu,2014,Exempt from Oversight,1,0,0,0.000000,0.000000,63000.0,ADVOCACIA-GERAL DA UNIAO,ADVOCACIA-GERAL DA UNIAO,63000.0,6042534.05,8.149639e+07,Advocacia-Geral da União,7478.500000,0.00,2428.666667,51.416667,448.916667,3306.250000,0.000000,48.166667,10708.750343,10122.291250,289.333333,20798.723403,NA - MDB - PV - PP - PSC - DEM - PT - PSDB - P...,None,PT,None,411.0,23.0,0.0,18.0,1.0,6.0,12.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,agu,2015,Exempt from Oversight,0,0,0,0.000000,0.000000,63000.0,ADVOCACIA-GERAL DA UNIAO,ADVOCACIA-GERAL DA UNIAO,63000.0,3543344.69,1.011973e+08,Advocacia-Geral da União,7520.416667,0.00,2452.416667,50.833333,433.500000,3216.333333,0.083333,48.250000,11615.547379,11362.767500,322.083333,22383.299752,NA - PV - PP - PSC - PT - PSDB - PTB - PDT - M...,NA - PP,PT,PP,510.0,39.0,1.0,20.0,0.0,0.0,10.0,5.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,agu,2016,Exempt from Oversight,4,2,2,0.002801,0.002801,63000.0,ADVOCACIA-GERAL DA UNIAO,ADVOCACIA-GERAL DA UNIAO,63000.0,4315913.71,4.583584e+07,Advocacia-Geral da União,7786.916667,0.00,2462.500000,52.416667,460.500000,3166.416667,1.000000,48.750000,12566.639531,14516.734167,352.166667,22858.325416,NA - PL - PSDB - PP - PSC - DEM - PV - REDE - ...,NA - PP,PT,PP,714.0,28.0,1.0,20.0,3.0,7.0,3.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,agu,2017,Exempt from Oversight,7,4,4,0.004994,0.004994,63000.0,ADVOCACIA-GERAL DA UNIAO,ADVOCACIA-GERAL DA UNIAO,63000.0,5634102.75,2.099410e+07,Advocacia-Geral da União,8045.583333,0.75,2606.000000,53.083333,466.416667,3231.666667,0.000000,51.416667,13164.154139,15237.513333,490.250000,24235.770912,NA - PL - PSDB - PP - PSC - PSL - PV - REDE - ...,None,PT,None,801.0,49.0,4.0,22.0,4.0,13.0,12.0,13.0,1.0,0.0,0.0,7.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [52]:
copy_df_painel_agency = df_painel_agency.copy()
copy_df_brazil_responses = df_brazil_responses.copy()

# Extracting only relevant columns of both
copy_df_painel_agency = copy_df_painel_agency[['sigla_new', 'year']]

desired_columns = [
    'Conceded Response',
    'Partially Conceded Response',
    'Denied',
    'Others',
    'Conceded and Partially conceded',
    'Total Requests',
    '% Conceded',
    '% Partially conceded',
    '% denied',
    '% Others',
    '% Concedico e Parcialmente',
]


copy_df_brazil_responses = copy_df_brazil_responses[['Agency', 'Year', *desired_columns]]
display(copy_df_painel_agency.head(5))
display(copy_df_brazil_responses.head(5))

print(f"Num of rows of df_brazil_responses: {len(df_brazil_responses.index)}")
print(f"Num of rows of df_painel_agency: {len(df_painel_agency.index)}")

# Standardizing agencies names in brazil_responses
# Getting the first value
copy_df_brazil_responses['Agency'] = copy_df_brazil_responses['Agency'].str.split().str[0]
# Lowering
copy_df_brazil_responses['Agency'] = copy_df_brazil_responses['Agency'].str.lower()

# Renaming columns for standardize
copy_df_brazil_responses = copy_df_brazil_responses.rename(columns={'Agency': 'sigla_new', 'Year': 'year'})
copy_df_brazil_responses.head(5)

# lowering values of painel_agency
copy_df_painel_agency['sigla_new'] = copy_df_painel_agency['sigla_new'].str.lower()

# Ensuring correct dtypes
copy_df_brazil_responses['year'] = copy_df_brazil_responses['year'].astype(int)
copy_df_painel_agency['year'] = copy_df_painel_agency['year'].astype(int)

num_matches = copy_df_painel_agency['sigla_new'].isin(copy_df_brazil_responses['sigla_new']).sum()
print(f"Num matches: {num_matches}")
copy_df_brazil_responses

,sigla_new,year
0,agu,2014
1,agu,2015
2,agu,2016
3,agu,2017
4,agu,2018


,Agency,Year,Conceded Response,Partially Conceded Response,Denied,Others,Conceded and Partially conceded,Total Requests,% Conceded,% Partially conceded,% denied,% Others,% Concedico e Parcialmente
0,ABGF - Agencia Brasileira Gestora de Fundos Ga...,2015.0,18.0,NaN,NaN,35.0,18,53,0.339623,0.000000,0.0,0.660377,0.339623
1,AEB – Agência Espacial Brasileira,2015.0,48.0,3.0,NaN,2.0,51,53,0.905660,0.056604,0.0,0.037736,0.962264
2,AMAZUL - Amazônia Azul Tecnologias de Defesa S.A.,2015.0,10.0,3.0,NaN,1.0,10,11,0.909091,0.000000,0.0,0.090909,0.909091
3,AN – Arquivo Nacional,2015.0,147.0,2.0,NaN,7.0,149,156,0.942308,0.012821,0.0,0.044872,0.955128
4,ANA – Agência Nacional de Águas,2015.0,318.0,1.0,NaN,1.0,319,320,0.993750,0.003125,0.0,0.003125,0.996875


Num of rows of df_brazil_responses: 3209
Num of rows of df_painel_agency: 1893
Num matches: 1488


,sigla_new,year,Conceded Response,Partially Conceded Response,Denied,Others,Conceded and Partially conceded,Total Requests,% Conceded,% Partially conceded,% denied,% Others,% Concedico e Parcialmente
0,abgf,2015,18.0,NaN,NaN,35.0,18,53,0.339623,0.000000,0.000000,0.660377,0.339623
1,aeb,2015,48.0,3.0,NaN,2.0,51,53,0.905660,0.056604,0.000000,0.037736,0.962264
2,amazul,2015,10.0,3.0,NaN,1.0,10,11,0.909091,0.000000,0.000000,0.090909,0.909091
3,an,2015,147.0,2.0,NaN,7.0,149,156,0.942308,0.012821,0.000000,0.044872,0.955128
4,ana,2015,318.0,1.0,NaN,1.0,319,320,0.993750,0.003125,0.000000,0.003125,0.996875
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3204,mt,2024,312.0,31.0,18.0,40.0,343,401,0.778055,0.077307,0.044888,0.099751,0.855362
3205,mte,2024,677.0,86.0,130.0,147.0,763,1040,0.650962,0.082692,0.125000,0.141346,0.733654
3206,mtur,2024,118.0,5.0,3.0,11.0,123,137,0.861314,0.036496,0.021898,0.080292,0.897810
3207,sgpr,2024,46.0,17.0,4.0,9.0,63,76,0.605263,0.223684,0.052632,0.118421,0.828947


In [56]:
import pandas as pd
from src.reading_data import RAW_DIR

df_merge = copy_df_painel_agency.merge(copy_df_brazil_responses, on=['sigla_new', 'year'], how='left')
display(df_merge)

df_final = df_painel_agency.merge(df_merge, on=['sigla_new', 'year'], how='left')
df_final.columns

def prepare_columns_to_excel(df: pd.DataFrame):
    # Dropping category column if it exists
    if 'category' in list(df.columns):
        df = df.drop(columns=['category'])
    
    # Changing names back to the original
    df.columns = df.columns.str.replace("_", " ")
    df.columns = df.columns.str.strip().str.replace("perc", "%", case=False)
    df.columns = df.columns.str.title()

    return df

def save_dataframe_to_excel_tab(df: pd.DataFrame, file_path: Path | str, sheet_name: str):
    # Save the DataFrame to the specific sheet
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

save_dataframe_to_excel_tab(df_final, RAW_DIR / 'painel-agency.xlsx', 'Sheet 2')

,sigla_new,year,Conceded Response,Partially Conceded Response,Denied,Others,Conceded and Partially conceded,Total Requests,% Conceded,% Partially conceded,% denied,% Others,% Concedico e Parcialmente
0,agu,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,agu,2015,150.0,1.0,21.0,123.0,151.0,295.0,0.508475,0.003390,0.071186,0.416949,0.511864
2,agu,2016,174.0,3.0,53.0,176.0,177.0,406.0,0.428571,0.007389,0.130542,0.433498,0.435961
3,agu,2016,107.0,3.0,25.0,79.0,107.0,211.0,0.507109,0.000000,0.118483,0.374408,0.507109
4,agu,2017,354.0,3.0,108.0,228.0,357.0,693.0,0.510823,0.004329,0.155844,0.329004,0.515152
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,valec,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2300,valec,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2301,valec,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2302,valec,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
